In [1]:
import os
from datetime import datetime
#BERT
import torch
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import  BertModel,BertForMaskedLM
import pickle
import numpy as np
import re

import threading

print(f'{datetime.now()} [ INFO  ] Finished imports, starting program.')


2020-06-11 13:59:14.866047 [ INFO  ] Finished imports, starting program.


In [2]:
# Data location
data_path = '/mnt/work/maiso/datasets/NYT'
print(f'{datetime.now()} [ INFO  ] Using {data_path} as the root data path.')

2020-06-11 13:59:15.993690 [ INFO  ] Using /mnt/work/maiso/datasets/NYT as the root data path.


In [3]:
print(f'{datetime.now()} [ INFO  ] Reading *.metadata filenames.')

## 1. Built the list with all the filenames:
files = []
for dirpath, _, filenames in os.walk(data_path):
    filenames = [filename for filename in filenames if filename.endswith('metadata')]
    for filename in filenames:
        file = os.path.join(dirpath, filename)
        files.append(file)

files.sort()
print(f'{datetime.now()} [ INFO  ] Finished reading *.metadata filenames from directory. Working with {len(files)} files.')

2020-06-11 13:59:16.745698 [ INFO  ] Reading *.metadata filenames.
2020-06-11 13:59:31.224261 [ INFO  ] Finished reading *.metadata filenames from directory. Working with 1855658 files.


In [4]:
xml_count = 0
metadata_count = 0
text_count = 0
lemmas_count = 0
events_count = 0
for dirpath, _, filenames in os.walk(data_path):
    xml_count += len([filename for filename in filenames if filename.endswith('.xml')])
    metadata_count += len([filename for filename in filenames if filename.endswith('metadata')])
    text_count += len([filename for filename in filenames if filename.endswith('.text')])
    lemmas_count += len([filename for filename in filenames if filename.endswith('.lemmas.p')])
    events_count += len([filename for filename in filenames if filename.endswith('.events.p')])
print(f'{datetime.now()} [ INFO  ] Curently are {xml_count} XMLs, {metadata_count} metadatas,{text_count} texts, {lemmas_count} lemmas and {events_count} event files.')

if events_count>0:
    print(f'{datetime.now()} [WARNING] Some files are goint to be overwritten (*.events.p)')

2020-06-11 13:59:43.626937 [ INFO  ] Curently are 1855658 XMLs, 1855658 metadatas,1855658 texts, 1855658 lemmas and 0 event files.


In [5]:
print(f'{datetime.now()} [ INFO  ] Loading BERT class')
class BERT_Embeddings(object):
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # Load pre-trained model (weights)
        self.model = BertModel.from_pretrained('bert-base-uncased')

        # Put the model in "evaluation" mode, meaning feed-forward operation.
        self.model.eval()


    def get_BERT_Embeddings(self,text, tokens):

            
        matrix = np.zeros(shape=(1,len(tokens),768)) # 768 last 4 layer added.
        



        tokenized_text = ['[CLS]']
        
        for idx, (ini,fin) in enumerate(tokens):
            word_tokens = self.tokenizer.tokenize(text[ini:fin])

            if len(word_tokens)+len(tokenized_text)+1 >512: #if sentence too long exclude some tokens
                break 
                
            tokenized_text += word_tokens
        
        # truncate sentence if is longer than 512 (BERT model can't handle more than 512)
        # if sentence too long exclude some tokens (the vectors of those tokens will be zero)
        tokens = tokens[:idx]

            
        tokenized_text += ['[SEP]']

            
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            encoded_layers, _ = self.model(tokens_tensor, segments_tensors)
            
        token_embeddings = [] 
        layer_i = 0
        batch_i = 0
        token_i = 0

        # For each token in the sentence...
        for token_i in range(len(tokenized_text)):
            hidden_layers = [] 

            # For each of the 12 layers...
            for layer_i in range(len(encoded_layers)):

                # Lookup the vector for `token_i` in `layer_i`
                vec = encoded_layers[layer_i][batch_i][token_i]

                hidden_layers.append(vec)

            token_embeddings.append(hidden_layers)

        summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]

        inicio=1
        for idx,token in enumerate(tokens):
                ini,fin = token
                individual_tokenized = self.tokenizer.tokenize(text[ini:fin])
                cantidad_embeddings = len(individual_tokenized)
                group_tokenized = tokenized_text[inicio:inicio+cantidad_embeddings]
                                
                assert all([t1==t2 for t1,t2 in zip(individual_tokenized,group_tokenized)]),  '{} {} {}'.format(
                                 text[ini:fin],
                                 individual_tokenized,
                                 group_tokenized
                             )

                if cantidad_embeddings>0:
                    for embedding in summed_last_4_layers[inicio:inicio+cantidad_embeddings]:
                        matrix[0,idx,:]+=embedding.numpy()
                    matrix[0,idx,:] = matrix[0,idx,:]/cantidad_embeddings
                    inicio+=cantidad_embeddings

        return matrix
print(f'{datetime.now()} [ INFO  ] BERT class loaded.')

2020-06-11 13:59:43.710614 [ INFO  ] Loading BERT class
2020-06-11 13:59:43.711439 [ INFO  ] BERT class loaded.


In [6]:
from modules.ed import get_data, load_model_from_disk, Generator, train_val_split, train_model, get_model, sensitivity, specificity, f1_score

Using TensorFlow backend.
/mnt/work/maiso/environments/causality/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/work/maiso/environments/causality/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/work/maiso/environments/causality/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/wor

In [7]:
from modules.ed import get_data, load_model_from_disk, Generator, train_val_split, train_model, get_model, sensitivity, specificity, f1_score

data_path='/mnt/work/maiso/python3.workspace/Causality/data/ed/'
model_path = '/mnt/work/maiso/python3.workspace/Causality/models/best_model_15.h5'

print(f'{datetime.now()} [ INFO  ] Loading data from files (testing_sents.p and training_sents.p) on folder: {data_path}')
      
    
x, y, ent2index = get_data(data_path)
train_data, val_data = train_val_split((x,y))

print(f'{datetime.now()} [ INFO  ] Loading model from: {model_path}')
# model = load_model_from_disk(model_path)
model = get_model(len(ent2index), hidden_units_count=15)

model.compile(loss='binary_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy',sensitivity,specificity, f1_score])

model.load_weights(model_path)

model._make_predict_function() # para read-only y evitar problemas de sincronización con multithreading

print(f'{datetime.now()} [ INFO  ] evaluating model on validation_data')
x_val, y_val = val_data
rta = model.evaluate(Generator(list(zip(x_val, y_val))), verbose=0)

# for metric, value in zip(['loss']+[m.name for m in model.metrics], rta):
#     print(f'{datetime.now()} [ INFO  ] {metric:12}: {value}')
for metric,value in zip(model.metrics_names, rta):
    print('{}: {:5.4f}'.format(metric, value))

2020-06-11 13:59:46.654614 [ INFO  ] Loading data from files (testing_sents.p and training_sents.p) on folder: /mnt/work/maiso/python3.workspace/Causality/data/ed/
[  OK   ] Loading training indexes.
2020-06-11 14:01:19.055195 [ INFO  ] Loading model from: /mnt/work/maiso/python3.workspace/Causality/models/best_model_15.h5




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-06-11 14:01:20.088195 [ INFO  ] evaluating model on validation_data
loss: 0.1672
acc: 0.9374
sensitivity: 0.6883
specificity: 0.9411
f1_score: 0.6916


In [41]:
# Inputs: 
bert = BERT_Embeddings()
# bert_lock = threading.Lock()

    
# model_lock = threading.Lock()
tuplas_re = re.compile('\(([0-9]*?), ([0-9]*?)\)')
entities_re = re.compile("'([^']*)'")

print(f'{datetime.now()} [ INFO  ] Loading BERT model.')
print(f'{datetime.now()} [ INFO  ] BERT model loaded.')

def process_file(file):



    # read file (metadata + text)
    text = open(file[:-9]+'.text', 'r' ).read()
    lemmas = pickle.load(open(file[:-9]+'.lemmas.p', 'rb' ))

    metadata = open(file,'r').read().splitlines()
    id_ = metadata[0][4:] #len('ID: ') == 4
    date = metadata[1][6:] # len('DATE: ') == 6

    # process file
    tokens = tuplas_re.findall(metadata[3])
    sentences = tuplas_re.findall(metadata[2])

    tokens = [(int(ini),int(fin)) for ini, fin in tokens]
    sentences = [(int(ini),int(fin)) for ini, fin in sentences]

    entities = entities_re.findall(metadata[4])

    idx = 0
    old_idx = 0
    sent_idx=0
    events = []

    for ini,fin in sentences:
        while idx<len(tokens) and tokens[idx][0]<fin:
            idx+=1
        tokens_in_sentence = tokens[old_idx:idx]
        lemmas_in_sentence = lemmas[old_idx:idx]
        entities_in_sentence = entities[old_idx:idx]
        old_idx = idx


        xbert = bert.get_BERT_Embeddings(text, tokens_in_sentence )

        xsentbert = np.average(xbert, axis=1) *np.ones(shape=(1,len(xbert[0,:,0]),768))
        xents = np.zeros(shape=(1,len(tokens_in_sentence)))


        for idx,entity in enumerate(entities_in_sentence):
            if entity in ent2index:
                xents[0,idx] = ent2index[entity]
            else:
                xents[0,idx] = ent2index['[PAD]']

        x = [xbert, xsentbert, xents]

        y_pred = model.predict(x)

        for token, pred in zip(tokens_in_sentence, y_pred[0,:,0]):
            if pred>0.5:
                print(f'event: {text[token[0]:token[1]]} - conf: {pred} - sent:{sent_idx}')
    
    
        for j in range(len(tokens_in_sentence)):
            pred = y_pred[0,j,0]
            if pred>0.5:
                bert_vec = xbert[0,j,:]
                token = tokens_in_sentence[j]


                event = {
                    'file_id':id_,
                    'trigger': text[token[0]:token[1]],
                    'idx': sent_idx,
                    'bert': bert_vec,
                    'conf': pred,
                    'lemma': lemmas_in_sentence[j],
                    'date': date
                }
                events.append(event)
        sent_idx+=1
    return events
#    pickle.dump(events ,open(file[:-9]+'.events.p', 'wb' ))



print(f'{datetime.now()} [ INFO  ] process_file defined. Uses: metadata, lemmas and text for building the event.p files.')

2020-06-11 14:48:03.341957 [ INFO  ] Loading BERT model.
2020-06-11 14:48:03.342254 [ INFO  ] BERT model loaded.
2020-06-11 14:48:03.343246 [ INFO  ] process_file defined. Uses: metadata, lemmas and text for building the event.p files.


In [42]:
import spacy
nlp = spacy.load('en_core_web_sm')
file = files[0]

events = process_file(file)


# text = open(file[:-9]+'.text', 'r' ).read()
# doc = nlp(text)
# for idx,sentence in enumerate(doc.sents):
#     print(f'{idx}: {sentence}')
    
for event in events:
    print(f"file_id: {event['file_id']} - trigger: {event['trigger']} - idx: {event['idx']} - lemma: {event['lemma']} - conf: {event['conf']} - date: {event['date']}")
    print('-----')
    print('-----')

event: LEAD - conf: 0.676846444606781 - sent:0
event: * - conf: 0.6415690779685974 - sent:2
event: * - conf: 0.6415690779685974 - sent:3
event: * - conf: 0.6840687394142151 - sent:4
event: * - conf: 0.6415690779685974 - sent:5
event: sales - conf: 0.6840687394142151 - sent:20
event: * - conf: 0.6415690779685974 - sent:31
event: * - conf: 0.6415690779685974 - sent:32
event: * - conf: 0.6840687394142151 - sent:33
event: * - conf: 0.6415690779685974 - sent:34
event: sales - conf: 0.6840687394142151 - sent:49
event: said - conf: 0.6840687394142151 - sent:61
event: outstanding - conf: 0.6840687394142151 - sent:61
event: offering - conf: 0.6840687394142151 - sent:61
file_id: 0000000 - trigger: LEAD - idx: 0 - lemma: lead - conf: 0.676846444606781 - date: 19870101
-----
-----
file_id: 0000000 - trigger: * - idx: 2 - lemma: * - conf: 0.6415690779685974 - date: 19870101
-----
-----
file_id: 0000000 - trigger: * - idx: 3 - lemma: * - conf: 0.6415690779685974 - date: 19870101
-----
-----
file_id:

In [32]:
len(events)

14

In [13]:
text = open(files[0][:-9]+'.text', 'r' ).read()
print(text)


LEAD:
*3*** COMPANY REPORTS **
*3*AAR CORP (NYSE)
Qtr to Nov 30
1986
1985
Sales
75,907,000
61,040,000
Net inc
3,953,000
2,858,000
Share earns
.38
.32
Shares outst
10,479,000
9,069,000
6mo sales
142,283,000
114,876,000
Net inc
7,054,000
5,300,000
Share earns
.71
.59
Shares outst
9,932,000
9,066,000
*3*** COMPANY REPORTS **
*3*AAR CORP (NYSE)
Qtr to Nov 30
1986
1985
Sales
75,907,000
61,040,000
Net inc
3,953,000
2,858,000
Share earns
.38
.32
Shares outst
10,479,000
9,069,000
6mo sales
142,283,000
114,876,000
Net inc
7,054,000
5,300,000
Share earns
.71
.59
Shares outst
9,932,000
9,066,000
The 1985 share earnings and shares outstanding are adjusted for the 3-for- 2 stock split
          in February 1986.
The company said the 1986 shares outstanding reflects the pro rata effect of the issuance
          of 1.35 million shares in public offering in August 1986.



In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [16]:
for idx,sentence in enumerate(doc.sents):
    print(f'{idx}: {sentence}')

0: 
LEAD:

1: *3
2: *
3: *
4: * COMPANY REPORTS
5: *
6: *

7: *3*AAR CORP (NYSE)

8: Qtr to Nov 30
1986

9: 1985

10: Sales
75,907,000

11: 61,040,000

12: Net inc

13: 3,953,000

14: 2,858,000

15: Share earns

16: .38

17: .32

18: Shares outst
10,479,000

19: 9,069,000

20: 6mo sales

21: 142,283,000

22: 114,876,000

23: Net inc

24: 7,054,000

25: 5,300,000

26: Share earns

27: .71

28: .59

29: Shares outst
9,932,000
9,066,000

30: *3
31: *
32: *
33: * COMPANY REPORTS
34: *
35: *

36: *3*AAR CORP (NYSE)

37: Qtr to Nov 30
1986

38: 1985

39: Sales
75,907,000

40: 61,040,000

41: Net inc

42: 3,953,000

43: 2,858,000

44: Share earns

45: .38

46: .32

47: Shares outst
10,479,000

48: 9,069,000

49: 6mo sales

50: 142,283,000

51: 114,876,000

52: Net inc

53: 7,054,000

54: 5,300,000

55: Share earns

56: .71

57: .59

58: Shares outst
9,932,000

59: 9,066,000

60: The 1985 share earnings and shares outstanding are adjusted for the 3-for- 2 stock split
          in February 1986

In [23]:
for event in events:
    print(f"file_id: {event['file_id']} - trigger: {event['trigger']} - idx: {event['idx']} - lemma: {event['lemma']} - conf: {event['conf']} - date: {event['date']}")
    print('-----')
    print('-----')

file_id: 0000000 - trigger: said - idx: 61 - lemma: say - conf: 0.6840687394142151 - date: 19870101
-----
-----
file_id: 0000000 - trigger: outstanding - idx: 61 - lemma: outstanding - conf: 0.6840687394142151 - date: 19870101
-----
-----
file_id: 0000000 - trigger: offering - idx: 61 - lemma: offering - conf: 0.6840687394142151 - date: 19870101
-----
-----


In [21]:
events[0].keys()

dict_keys(['file_id', 'trigger', 'idx', 'bert', 'conf', 'lemma', 'date'])

In [22]:
events[0]['conf']

0.68406874